<a href="https://colab.research.google.com/github/phgnhug/LION_DEN/blob/Rachel-Branch/trial2_DATA_CLEANING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name map


---


trial -> trying new variable

---


data_cleaning -> a separate file which ends with saving a pickle that you use in model.ipynb


---


model -> Main model with little extra code

---


model_testing -> Less code, optimized for testing

---


In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import numpy as np
file_path= "/content/austinHousingData.csv"
data = pd.read_csv(file_path)

In [ ]:
data.columns

Index(['zpid', 'city', 'streetAddress', 'zipcode', 'description', 'latitude',
       'longitude', 'propertyTaxRate', 'garageSpaces', 'hasAssociation',
       'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa', 'hasView',
       'homeType', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'latestPriceSource', 'numOfPhotos',
       'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWaterfrontFeatures', 'numOfWindowFeatures',
       'numOfCommunityFeatures', 'lotSizeSqFt', 'livingAreaSqFt',
       'numOfPrimarySchools', 'numOfElementarySchools', 'numOfMiddleSchools',
       'numOfHighSchools', 'avgSchoolDistance', 'avgSchoolRating',
       'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBathrooms',
       'numOfBedrooms', 'numOfStories', 'homeImage'],
      dtype='object')

In [ ]:
################ NEW VARIABLE TEST ######### TO MERGE ON ZIP ##############################

In [12]:
house_inventory = pd.read_csv("/content/Affordable_Housing_Inventory_20250326.csv")
house_inventory.columns

Index(['Contract_ID', 'Project_ID', 'Project_Name', 'Ownership', 'Developer',
       'Address', 'ZIP', 'Parcel_ID', 'Council_District', 'Unit_Type',
       'Housing_Tenure', 'Development_Status', 'Total_Units',
       'Affordable_Units', 'Total_ADA_Units', 'PSH_Units', 'MFI_20', 'MFI_30',
       'MFI_40', 'MFI_50', 'MFI_60', 'MFI_65', 'MFI_70', 'MFI_80', 'MFI_100',
       'MFI_120', 'Affordability_Start_Date', 'Affordability_Expiration_Date',
       'Affordability_Period', 'FIL_Status', 'CAL_FIL', 'Date_FIL_Recorded',
       'HDA', 'Private_Activity_Bonds', 'Affordability_Unlocked', 'SMART',
       'Dev_Agreement', 'Density_Bonus', 'FIL', 'LIHTC', 'GO_Bonds_2006',
       'GO_Bonds_2013', 'GO_Bonds_2018', 'GO_Bonds_2022', 'CDBG', 'CIP_Funds',
       'HOME', 'HTF', 'Project_Connect', 'City_Subsidy', 'Longitude',
       'Latitude'],
      dtype='object')

In [13]:
########### CHECKING IF MERGING IS RIGHT #########
missing_count =(data['zipcode'].isin(house_inventory['ZIP']) == False).sum()
print(missing_count)


2032


if we have n unique values in one of the columns in the new data set


In [14]:
house_inventory['ZIP_int'] = house_inventory['ZIP'].dropna().astype(int)
house_inventory['ZIP_int']

,ZIP_int
0,78722.0
1,78745.0
2,78704.0
3,78704.0
4,78704.0
...,...
2370,78744.0
2371,7857.0
2372,78723.0
2373,78653.0


In [15]:
############### DUPLICATED LONGITUDE AND LATITUDE IN HOUSE_INVENTORY ################## FATAL ##########
lat_long_counts = house_inventory.groupby(['Latitude', 'Longitude']).size().reset_index(name='count')
duplicates = lat_long_counts[lat_long_counts['count'] > 1]
print(duplicates)

       Latitude  Longitude  count
0     30.089704 -97.791678      2
4     30.118147 -97.810538     24
7     30.124392 -97.807296      4
119   30.140566 -97.765717      2
207   30.153442 -97.753764      6
...         ...        ...    ...
1919  30.336559 -97.707100      6
1920  30.336560 -97.707100     13
1927  30.340805 -97.739455      3
1960  30.364212 -97.726877      2
2010  30.400555 -97.717067      2

[118 rows x 3 columns]


In [16]:
house_inventory_unique = house_inventory.drop_duplicates(subset='ZIP', keep='first')
print(house_inventory_unique.shape)

(45, 53)


In [17]:
data= data[data['city'] == 'austin']

In [ ]:
################# MOCK MERGE , BECAUSE DUPLICATED ZIPS HAVE DIFFERENT CROSS SECTIONAL DATA, WE ARE ONLY TAKING THE FIRST DUPLICATE
############### IMPLICATION : THE DATA IS A REPRESENTATION OF WHAT TYPE OF PROPERTY IS AT THAT ZIPCODE,
######### WE ARE NOT TAKING INTO ACCOUNT OTHER PROPERTIES IN THAT ZIPCODE BUT ONLY THE FIRST

data['Housing_Tenure'] = data['zipcode'].map(
    house_inventory_unique.set_index('ZIP')['Housing_Tenure']
)
print(data.shape)
print(data.isna().sum())

(15020, 48)
zpid                             0
city                             0
streetAddress                    0
zipcode                          0
description                      2
latitude                         0
longitude                        0
propertyTaxRate                  0
garageSpaces                     0
hasAssociation                   0
hasCooling                       0
hasGarage                        0
hasHeating                       0
hasSpa                           0
hasView                          0
homeType                         0
parkingSpaces                    0
yearBuilt                        0
latestPrice                      0
numPriceChanges                  0
latest_saledate                  0
latest_salemonth                 0
latest_saleyear                  0
latestPriceSource                0
numOfPhotos                      0
numOfAccessibilityFeatures       0
numOfAppliances                  0
numOfParkingFeatures             0
numOfPat

In [18]:
#USELESS FEATURES ## WE DROP USED FEATURES ONLY, "city" (used) and "zipcode" (unused) .
useless_features= ['zpid', 'city', 'streetAddress', 'description','homeImage','homeType', 'latestPriceSource']

In [22]:
################# FINAL DATAFRAME IN USE ################
data = data.drop(columns=useless_features, errors='ignore')
data.isna().sum()

,0
zipcode,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0


In [23]:
####### WE DROP AFTER REMOVING USELESS ###
data = data.dropna()
data.shape

(15020, 40)

In [24]:
############ FEATURE ENGINEERING ################
import pandas as pd
def create_dummies(df, column):
  if df[column].nunique() <= 7 :
    return pd.get_dummies(df[column], prefix= column, dtype=int)
  else:
      return df[[column]]

dummies = create_dummies(data, 'Housing_Tenure')
data= data.join(dummies)
data

KeyError: 'Housing_Tenure'

In [25]:
data.select_dtypes(exclude='number').columns

Index(['hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'latest_saledate'],
      dtype='object')

In [26]:
zp = pd.read_csv("/content/uszips.csv")
zp['zip']

,zip
0,601
1,602
2,603
3,606
4,610
...,...
33778,99923
33779,99925
33780,99926
33781,99927


In [27]:
zp[zp['city']== 'Austin']['zip'].nunique()

51

In [28]:
########### CHECKING IF MERGING IS RIGHT #########
missing_count = (data['zipcode'].isin(zp['zip']) == False).sum()
print(missing_count)

0


In [29]:
import pandas as pd
zp = zp.set_index('zip')
zp.head()

,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
zip,,,,,,,,,,,,,,,,,
601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16721.0,100.2,72001,Adjuntas,"{""72001"": 98.74, ""72141"": 1.26}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37510.0,477.6,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,48317.0,543.1,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5435.0,47.3,72093,Maricao,"{""72093"": 82.27, ""72153"": 11.66, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,25413.0,264.4,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.82, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


In [30]:
data['population'] = data['zipcode'].map(zp['population'])
data['density'] = data['zipcode'].map(zp['density'])

In [31]:
data[data['population'].isna()]['population'].sum()

np.float64(0.0)

In [32]:
#USELESS FEATURES ## BUT WE DROP "city" and "zipcode" after using them. ## no dropping yet
useless_features_2= ['zipcode','zpid', 'city', 'streetAddress', 'description','homeImage','homeType', 'latestPriceSource']

In [33]:
df_1 = data.drop(columns=useless_features_2, errors= 'ignore')
df_1.isna().sum()

,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0
hasView,0


In [34]:
df_1['latest_saledate'] = df_1['latest_saledate'].astype(str).str.strip()
df_1['latest_saledate'] = pd.to_datetime(df_1['latest_saledate'], errors='coerce')
print(df_1['latest_saledate'].isna().sum())

0


<ipython-input-34-bc364721aef3>:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_1['latest_saledate'] = pd.to_datetime(df_1['latest_saledate'], errors='coerce')


In [35]:
df_1['latest_saledate'].head()

,latest_saledate
17,2018-08-28
18,2018-12-03
19,2020-12-22
20,2020-01-10
21,2020-12-07


In [36]:
df_1 = df_1.sort_values('latest_saledate').reset_index(drop=True)
df_1['latest_saledate']

,latest_saledate
0,2018-01-22
1,2018-01-22
2,2018-01-22
3,2018-01-22
4,2018-01-23
...,...
15015,2021-01-27
15016,2021-01-28
15017,2021-01-29
15018,2021-01-29


In [37]:
 # We have 958 days of analysis
 df_1['latest_saledate'].value_counts().sort_index().isna().sum()

np.int64(0)

In [ ]:
df_1.select_dtypes(include= object)


,Housing_Tenure
0,Rental
1,Rental
2,Rental
3,Rental
4,Rental
...,...
13000,Rental
13001,Ownership
13002,Ownership
13003,Rental


In [38]:
# FINAL CHECK BEFORE MERGING
df_1.isna().sum()

,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0
hasView,0


In [39]:
# Get Mortgage rate
import pandas as pd
MTG = pd.read_csv('/content/MORTGAGE30US.csv')

print(MTG.shape)

MPPSQ = pd.read_csv('/content/MEDLISPRIPERSQUFEE12420.csv')
print(MPPSQ.dtypes)

(160, 2)
observation_date           object
MEDLISPRIPERSQUFEE12420     int64
dtype: object


In [ ]:
############################################################################### Tranforming Mortgage Data #############################################################################################

In [40]:
################## TREATING DATES ON EXTERNAL MACRO VARIABLES ##################
def parse_obs_date(date_str):
    d = str(date_str).strip().split()[0]
    if "/" in d:
        return pd.to_datetime(d, format='%d/%m/%Y', errors='raise')
    else:
        return pd.to_datetime(d, errors='raise')

MTG['observation_date'] = MTG['observation_date'].apply(parse_obs_date)
MPPSQ['observation_date'] = MPPSQ['observation_date'].apply(parse_obs_date)
MTG['MORTGAGE30US'].dtypes

dtype('float64')

In [41]:
print(MTG.isna().sum())
print(MPPSQ.isna().sum())

observation_date    0
MORTGAGE30US        0
dtype: int64
observation_date           0
MEDLISPRIPERSQUFEE12420    0
dtype: int64


In [42]:
df_2 = df_1.drop(columns='Housing_Tenure', errors= 'ignore')
df_2

,latitude,longitude,propertyTaxRate,garageSpaces,hasAssociation,hasCooling,hasGarage,hasHeating,hasSpa,hasView,...,numOfHighSchools,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories,population,density
0,30.197430,-97.741470,1.98,0,False,True,False,True,False,False,...,1,1.066667,3.000000,794,12,2.0,3,1,51069.0,872.3
1,30.157705,-97.782372,1.98,2,True,True,True,True,False,False,...,1,2.633333,5.333333,1057,14,3.0,3,2,51069.0,872.3
2,30.163763,-97.940430,2.01,0,True,True,False,True,False,False,...,0,3.900000,8.000000,743,15,3.0,4,1,26413.0,279.9
3,30.443314,-97.802513,1.98,0,True,True,False,True,False,False,...,1,0.866667,8.666667,1632,16,3.0,4,1,29322.0,959.6
4,30.265963,-97.704697,1.98,1,False,True,True,True,False,False,...,1,1.000000,5.333333,679,14,3.0,3,2,26495.0,2043.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15015,30.244835,-97.761124,1.98,2,False,True,True,True,False,False,...,1,0.800000,5.000000,889,13,3.0,3,2,50479.0,2223.8
15016,30.367903,-97.653358,1.98,4,True,True,True,True,True,False,...,2,3.600000,4.333333,1066,12,3.0,4,2,31410.0,801.1
15017,30.424355,-97.726814,1.98,2,False,True,True,True,True,False,...,1,2.966667,6.666667,1486,15,2.0,3,1,30894.0,1535.4
15018,30.357496,-97.642845,1.98,0,True,True,False,True,False,False,...,2,2.833333,4.333333,1066,12,2.0,4,1,31410.0,801.1


In [43]:
##################################.MERGE. ####################################

import pandas as pd

df_2['latest_saledate'] = pd.to_datetime(df_2['latest_saledate'], errors='coerce')
df_2 = df_2.sort_values('latest_saledate')

# Read and prepare the MTG rates
MTG['observation_date'] = pd.to_datetime(MTG['observation_date'], format='%d/%m/%Y', errors='coerce')
MTG = MTG.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MTG rate available
df_2 = pd.merge_asof(df_2, MTG, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'population',
       'density', 'MORTGAGE30US'],
      dtype='object')

In [44]:
# read variable, sort variable on date
MPPSQ['observation_date'] = pd.to_datetime(MPPSQ['observation_date'], format='%d/%m/%Y', errors='coerce')
MPPSQ = MPPSQ.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MPPSQ available
df_2 = pd.merge_asof(df_2, MPPSQ, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if you no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'population',
       'density', 'MORTGAGE30US', 'MEDLISPRIPERSQUFEE12420'],
      dtype='object')

In [45]:
df_2[['latest_saledate','MORTGAGE30US','MEDLISPRIPERSQUFEE12420']]

,latest_saledate,MORTGAGE30US,MEDLISPRIPERSQUFEE12420
0,2018-01-22,4.04,163
1,2018-01-22,4.04,163
2,2018-01-22,4.04,163
3,2018-01-22,4.04,163
4,2018-01-23,4.04,163
...,...,...,...
15015,2021-01-27,2.77,226
15016,2021-01-28,2.73,226
15017,2021-01-29,2.73,226
15018,2021-01-29,2.73,226


In [46]:
df_2.shape

(15020, 43)

In [47]:
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'population',
       'density', 'MORTGAGE30US', 'MEDLISPRIPERSQUFEE12420'],
      dtype='object')

In [48]:
df_2['latestPrice_log'] = np.log(df_2['latestPrice'])
df_2.shape

(15020, 44)

In [49]:
df_2 = df_2.drop(columns= 'latestPrice', errors='ignore')
df_2.shape

(15020, 43)

In [50]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
# change path as desired, use the same path in the model file
df_2.to_pickle("/content/drive/MyDrive/df_2_trial2.pkl")

In [64]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
!ls /content/drive/MyDrive/


 0B8BDA76-02AC-4518-BA12-BA217131E76C.png
 122625_Nguyen-1.docx
 A_L8.docx.docx
'Analyst Intern.pdf'
 austinHousingData.csv
'Bảng tính chưa có tiêu đề.gsheet'
'Bản sao của A_L8.docx.docx'
'Bản sao của english II.gdoc'
'Bản sao của Marketing Questions (1).gdoc'
'Bản sao của Marketing Questions.gdoc'
'Bản sao của micro practice.gdoc'
'Case study_Handshake report.docx'
'Câu hỏi Marketing .gdoc'
'Chap 1 Sociology .gdoc'
'Colab Notebooks'
'ddaps and.gdoc'
'Design T2-T3.docx'
'Design T2-T3.gdoc'
 df_2_trial2.pkl
'dokument_36103 (2).pdf'
'Được lưu từ Chrome'
 English.gdoc
 exercises.tar.7z
 exercises.tar.gz
 exercises.tar.tar.bz2
'File đáp án Sociology.gdoc'
'Hand shake.pdf'
 Homework.docx
 Homework.gdoc
'Hợp đồng hợp tác kinh doanh 2023.gdoc'
'Interview aiesec.gdoc'
 iwona-stempek-krok-po-kroku-polski-a1-2010.pdf
'LECTURE I .gdoc'
 MEDLISPRIPERSQUFEE12420.csv
 MORTGAGE30US.csv
'Nguyen Thi Phuong Nhung.gdoc'
'Nhung Nguyen 07072024.pdf'
'Nhung Nguyen (1).pdf'


In [ ]:
## run -> on next script -> df_2 = pd.read_pickle("/content/drive/MyDrive/lions_den_data/MODEL & DATA_CLEANING/df_2_trial2.pkl")
